In [3]:
import os
print(os.getcwd())

/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/notebooks


In [4]:
import os
print(os.listdir('../data/raw'))

['ASD_9-1_test.csv']


In [5]:
import pandas as pd

# 注意：前面加上 ../ 回到專案根目錄
df = pd.read_csv('../data/raw/ASD_9-1_test.csv')
df.head()

,tmdbid,Company,Production_Series,Production_No,收件日期,損壞材料品牌,損壞材料型號,損壞材料料號,對策狀態,對策狀態描述,Issue_Level_1_客訴故障現象,Issue_Level_2_分析看到現象,Issue_Level_3_廠內測試項目_Series,Issue_Level_4_故障板類_Series,Issue_Level_5_Function_Block_Series,Issue_Level_6_故障零件位置,Issue_Level_7_損壞原因,Issue_Level_8_損壞原因二,Issue_Level_9_責任歸屬
0,3,M公司,系列A,系列A-0421-L,1/22/24,Txx,Txx62569DBVR,料號2510768214,A已導入,"""1.ECN更換Txx62565（2510726214）---1/13 ECN 126A-2...",無,帶馬達運轉正常,T1,Can Not Duplicate,Can Not Duplicate,無,帶馬達運轉正常,無法重現不良,CND
1,6,T公司,系列A,系列A-0421-M,1/22/24,Txx,Txx62569DBVR,料號2510768214,A已導入,"""1.ECN更換Txx62565（2510726214）---1/13 ECN 126A-2...",上電無顯示,帶馬達運轉正常,T1,Power Board,INVERTER,"IGBT,DR58,DR57",IGBT損壞,原因不明,功率模組
2,9,B公司,系列A,系列A-1021-F,1/22/24,Txx,Txx62569DBVR,料號2510768214,A已導入,"""1.ECN更換Txx62565（2510726214）---1/13 ECN 126A-2...",驱动器自带了CN10，报警AL500,裝STO端子后帶馬達運轉正常,T1,Power Board,"Internal Connector (J1, DJ2, EJ3…)",DJ1,PIN30 外漏,製程-組立,製程問題
3,12,C公司,系列A,系列A-0421-L,1/22/24,Txx,Txx320F28374DPTPT,料號2611309608,A已導入,1.優化測具探針Over travel行程參數，由原來的110μm變更為90μm (2020...,电压正常时一直报电压过低,V-BUS電壓異常為12V,T1,Can Not Duplicate,Can Not Duplicate,無,帶馬達運轉正常,無法重現不良,CND
4,15,M公司,系列A,系列A-0421-L,1/22/24,无失效现象,无失效现象,料號无失效现象,A已導入,"""1.ECN更換Txx62565（2510726214）---1/13 ECN 126A-2...",按键失灵,USB无法通讯,T1,Can Not Duplicate,Can Not Duplicate,無,帶馬達運轉正常,無法重現不良,CND


In [6]:
# 假設日期欄位名稱為 'date'，請依實際欄位名稱替換
date_col = '收件日期'  # ←請改成你的日期欄位名稱

# 觀察前幾筆資料
print(df[date_col].head(10))

# 列出所有不同的日期格式（用 set 去重複）
unique_formats = set()
for val in df[date_col].dropna():
    unique_formats.add(val)
print(f"共有 {len(unique_formats)} 種不同的日期字串：")
for v in list(unique_formats)[:20]:  # 只印前20種，避免太多
    print(v)

0    1/22/24
1    1/22/24
2    1/22/24
3    1/22/24
4    1/22/24
5    1/10/24
6    1/19/24
7    1/22/24
8    1/18/24
9    1/15/24
Name: 收件日期, dtype: object
共有 224 種不同的日期字串：
6/13/24
4/10/24
1/18/25
1/9/24
4/17/24
9/11/24
8/14/24
12/26/24
7/12/24
7/19/24
1/2/24
1/14/25
3/6/24
4/3/24
1/29/24
3/1/24
7/15/24
8/29/24
12/17/24
4/2/24


In [7]:
# 假設日期欄位名稱為 'date'，請依實際欄位名稱替換
date_col = '收件日期'  # ←請改成你的日期欄位名稱

# 轉換為 datetime 格式
df[date_col] = pd.to_datetime(df[date_col], format='%m/%d/%y')

# 檢查結果
df[date_col].head()

0   2024-01-22
1   2024-01-22
2   2024-01-22
3   2024-01-22
4   2024-01-22
Name: 收件日期, dtype: datetime64[ns]

In [8]:
# 假設欄位名稱為 'Issue_Level_6_故障零件位置'
col = 'Issue_Level_6_故障零件位置'

def normalize_parts(parts_str):
    if pd.isna(parts_str):
        return parts_str
    # 以逗號分隔，去除空白，排序後再合併
    parts = [p.strip() for p in parts_str.split(',')]
    parts_sorted = sorted(parts)
    return ','.join(parts_sorted)

df[col + '_正規化'] = df[col].apply(normalize_parts)

# 檢查結果
df[[col, col + '_正規化']].head(10)


,Issue_Level_6_故障零件位置,Issue_Level_6_故障零件位置_正規化
0,無,無
1,"IGBT,DR58,DR57","DR57,DR58,IGBT"
2,DJ1,DJ1
3,無,無
4,無,無
5,無,無
6,"DR58,DPM1","DPM1,DR58"
7,DPM1,DPM1
8,DD22，IGBT，PCB板,DD22，IGBT，PCB板
9,DPM1,DPM1


In [9]:
# 將處理後的資料存到 data/cleaned/ 資料夾
df.to_csv('/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/data/cleaned/ASD_9-1_test_cleaned.csv', index=False)


In [10]:
!pip install opencc-python-reimplemented pandas
!pip install opencc-python-reimplemented
from opencc import OpenCC

In [11]:
# 建立簡體轉繁體的轉換器
cc = OpenCC('s2t')  # s2t: Simplified Chinese to Traditional Chinese

# 將所有欄位的內容進行轉換
def convert_to_traditional(text):
    if isinstance(text, str):
        return cc.convert(text)
    return text

df = df.applymap(convert_to_traditional)

# 儲存為新的 CSV
#df.to_csv('output.csv', index=False, encoding='utf-8')
# 將處理後的資料存到 data/cleaned/ 資料夾
df.to_csv('/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/data/cleaned/ASD_9-1_test_cleaned.csv', index=False)


/var/folders/w6/s2_dqw991_jdj_vjn8y72n3r0000gq/T/ipykernel_3083/2073879568.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convert_to_traditional)


In [12]:
# 統計包含「螺絲」的客訴故障現象
col = 'Issue_Level_1_客訴故障現象'

# 篩選包含「螺絲」的資料
screw_related = df[df[col].astype(str).str.contains('螺絲', na=False)]

# 統計出現次數
screw_counts = screw_related[col].value_counts()

print("與螺絲相關的客訴故障現象統計：")
print(screw_counts)


與螺絲相關的客訴故障現象統計：
Issue_Level_1_客訴故障現象
螺絲滑牙                        54
螺絲滑絲                         8
接地螺絲擰不緊                      5
螺絲擰不緊                        4
其他(螺絲滑牙)&                    4
                            ..
r項螺絲滑牙                       1
其他(新機，客戶打開包裝後自機器本體內掉出螺絲)     1
庫存機拿出測試發現螺絲滑牙                1
第二個螺絲滑牙，擰不出                  1
R螺絲滑牙，上電報警                   1
Name: count, Length: 65, dtype: int64


In [20]:
# 將統計結果轉換為 DataFrame
screw_counts_df = screw_counts.reset_index()
screw_counts_df.columns = [col, '出現次數']

# 輸出到 CSV
output_path = '/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/data/cleaned/screw_counts.csv'
screw_counts_df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [19]:
# ...existing code...
!pip install openpyxl
# 篩選包含「螺絲」的客訴故障現象
col = 'Issue_Level_1_客訴故障現象'
screw_related = df[df[col].astype(str).str.contains('螺絲', na=False)]

# 輸出到 Excel
output_path = '/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/data/cleaned/screw_related_issues.csv'
screw_related.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"已將與螺絲相關的資料輸出到：{output_path}")

已將與螺絲相關的資料輸出到：/Users/alvinliu_mbp/Downloads/Alvin/Projects/kg-data-cleaning/python-data-cleaning/data/cleaned/screw_related_issues.csv
